In [1]:
import numpy as np
import pandas as pd
import scipy.linalg

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
from matplotlib.patches import Ellipse
import matplotlib.transforms as transforms

import sys
print(sys.version)
sys.path.append('../')
import os
import copy

from models.powertrain.bounded_powertrain import Bounded_powertrain
from models.kinematic.ideal_diff_drive import Ideal_diff_drive
from models.learning.gp_slip import *
from models.kinematic.ICR_based import *
from models.kinematic.Perturbed_unicycle import *
from models.kinematic.enhanced_kinematic import *


from util.transform_algebra import *
from util.util_func import *

from sklearn import linear_model

3.10.6 (main, Mar 10 2023, 10:55:28) [GCC 11.3.0]


In [2]:
# import slip dataset

# dataset_path = '../data/ral2023_dataset/husky/boreal_snow/slip_dataset_all.pkl'
# dataset_path = '../data/ral2023_dataset/husky/grand_salon_tile_inflated/slip_dataset_all.pkl'
# dataset_path = '../data/ral2023_dataset/husky/grand_salon_left-deflated/slip_dataset_all.pkl'
# dataset_path = '../data/ral2023_dataset/marmotte/boreal_snow/slip_dataset_all.pkl'
# dataset_path = '../data/ral2023_dataset/marmotte/ga_hard_snow_a/slip_dataset_all.pkl'
# dataset_path = '../data/ral2023_dataset/marmotte/ga_hard_snow_b/slip_dataset_all.pkl'
# dataset_path = '../data/ral2023_dataset/marmotte/grand_salon_tile_b/slip_dataset_all.pkl'
dataset_path = '../data/ral2023_dataset/warthog_tracks/boreal_mud/slip_dataset_all.pkl'
# dataset_path = '../data/ral2023_dataset/warthog_tracks/grand-axe_crusted-snow/slip_dataset_all.pkl'


full_dataset = pd.read_pickle(dataset_path)
full_dataset_length = len(full_dataset)
train_dataset = full_dataset[:int(full_dataset_length/2)]
validate_dataset = full_dataset[int(full_dataset_length/2):]

print(train_dataset)

     init_icp_x  init_icp_y  init_icp_z  init_icp_roll  init_icp_pitch  \
0           0.0         0.0         0.0            0.0             0.0   
1           0.0         0.0         0.0            0.0             0.0   
2           0.0         0.0         0.0            0.0             0.0   
3           0.0         0.0         0.0            0.0             0.0   
4           0.0         0.0         0.0            0.0             0.0   
..          ...         ...         ...            ...             ...   
185         0.0         0.0         0.0            0.0             0.0   
186         0.0         0.0         0.0            0.0             0.0   
187         0.0         0.0         0.0            0.0             0.0   
188         0.0         0.0         0.0            0.0             0.0   
189         0.0         0.0         0.0            0.0             0.0   

     init_icp_yaw  calib_step  cmd_left_0  cmd_right_0  cmd_left_1  ...  \
0             0.0         0.0   -9.2

In [3]:
# isolate steady-state data

# extract cmd_body_vel arrays (input arrays)

idd_body_vel_x_str_list = []
idd_body_vel_y_str_list = []
idd_body_vel_yaw_str_list = []
for i in range(0, 40):
    str_idd_vel_x_i = 'idd_vel_x_' + str(i)
    str_idd_vel_y_i = 'idd_vel_y_' + str(i)
    str_idd_vel_yaw_i = 'idd_vel_yaw_' + str(i)
    idd_body_vel_x_str_list.append(str_idd_vel_x_i)
    idd_body_vel_y_str_list.append(str_idd_vel_y_i)
    idd_body_vel_yaw_str_list.append(str_idd_vel_yaw_i)
idd_body_vel_x_array = train_dataset[idd_body_vel_x_str_list].to_numpy()
idd_body_vel_y_array = train_dataset[idd_body_vel_y_str_list].to_numpy()
idd_body_vel_yaw_array = train_dataset[idd_body_vel_yaw_str_list].to_numpy()

x_train = np.column_stack((idd_body_vel_x_array.flatten(), idd_body_vel_yaw_array.flatten()))

# extract icp vels

str_icp_vel_x_list = []
str_icp_vel_y_list = []
str_icp_vel_yaw_list = []
for i in range(0, 40):
    str_icp_vel_x_i = 'icp_vel_x_' + str(i)
    str_icp_vel_y_i = 'icp_vel_y_' + str(i)
    str_icp_vel_yaw_i = 'icp_vel_yaw_' + str(i)
    str_icp_vel_x_list.append(str_icp_vel_x_i)
    str_icp_vel_y_list.append(str_icp_vel_y_i)
    str_icp_vel_yaw_list.append(str_icp_vel_yaw_i)
icp_vel_x_array = train_dataset[str_icp_vel_x_list].to_numpy()
icp_vel_y_array = train_dataset[str_icp_vel_y_list].to_numpy()
icp_vel_yaw_array = train_dataset[str_icp_vel_yaw_list].to_numpy()

# extract body_vel_distruptions arrays (output arrays)

str_body_vel_disturption_x_list = []
str_body_vel_disturption_y_list = []
str_body_vel_disturption_yaw_list = []
for i in range(0, 40):
    str_body_vel_disturption_x_i = 'body_vel_disturption_x_' + str(i)
    str_body_vel_disturption_y_i = 'body_vel_disturption_y_' + str(i)
    str_body_vel_disturption_yaw_i = 'body_vel_disturption_yaw_' + str(i)
    str_body_vel_disturption_x_list.append(str_body_vel_disturption_x_i)
    str_body_vel_disturption_y_list.append(str_body_vel_disturption_y_i)
    str_body_vel_disturption_yaw_list.append(str_body_vel_disturption_yaw_i)
    
body_vel_disturption_x_array = train_dataset[str_body_vel_disturption_x_list].to_numpy()
body_vel_disturption_y_array = train_dataset[str_body_vel_disturption_y_list].to_numpy()
body_vel_disturption_yaw_array = train_dataset[str_body_vel_disturption_yaw_list].to_numpy()

y_train_longitudinal_slip = body_vel_disturption_x_array.flatten()
y_train_lateral_slip = body_vel_disturption_y_array.flatten()
y_train_angular_slip = body_vel_disturption_yaw_array.flatten()

# compute mean body vel disturbance for each steady-state window
n_windows = len(train_dataset)

steady_state_mask = train_dataset['steady_state_mask'].to_numpy() == True

steady_state_idd_body_vel_x = idd_body_vel_x_array[steady_state_mask]
steady_state_idd_body_vel_y = idd_body_vel_y_array[steady_state_mask]
steady_state_idd_body_vel_yaw = idd_body_vel_yaw_array[steady_state_mask]

steady_state_icp_body_vel_x = icp_vel_x_array[steady_state_mask]
steady_state_icp_body_vel_y = icp_vel_y_array[steady_state_mask]
steady_state_icp_body_vel_yaw = icp_vel_yaw_array[steady_state_mask]

steady_state_body_vel_disturption_x = body_vel_disturption_x_array[steady_state_mask]
steady_state_body_vel_disturption_y = body_vel_disturption_y_array[steady_state_mask]
steady_state_body_vel_disturption_yaw = body_vel_disturption_yaw_array[steady_state_mask]

In [4]:
# import GP model


length_scales_x = [0.1]
length_scales_y = [0.1]
length_scales_yaw = [100]
noise_levels_x = 0.1
noise_levels_y = 0.1
noise_levels_yaw = 0.5

n_restarts_optimizer = 20

# robot = 'husky'
# robot = 'marmotte'
robot = 'warthog-track'
if robot=='husky':
    wheel_radius = 0.33 / 2
    baseline = 0.55
    dt=0.05
if robot=='marmotte':
    wheel_radius = 0.116
    baseline = 0.5927
    dt=0.05
if robot == 'warthog-track':
    wheel_radius = 0.3
    baseline = 1.1652
    dt = 0.05
kappa_param = 1

full_body_slip_gp = FullBodySlipGaussianProcess(1, 1, 3, length_scales_x, length_scales_y, length_scales_yaw,
                 noise_levels_x, noise_levels_y, noise_levels_yaw, n_restarts_optimizer, baseline, wheel_radius, dt, kappa_param)

In [5]:
# train params

full_body_slip_gp.train_params(steady_state_idd_body_vel_x, steady_state_idd_body_vel_yaw, 
                               steady_state_body_vel_disturption_x, steady_state_body_vel_disturption_y, steady_state_body_vel_disturption_yaw, True)

full_body_slip_gp.save_params('../data/ral2023_dataset/warthog_tracks/boreal_mud/trained_params/slip/gp/steady-state/')

gpr_x
0.7655445670010352
0.623**2 * RBF(length_scale=1.25) + WhiteKernel(noise_level=0.0082)
gpr_y
0.7933326513974224
0.439**2 * RBF(length_scale=5.58) + WhiteKernel(noise_level=0.000641)
gpr_yaw
0.9963399676177387
2.79**2 * RBF(length_scale=3.28) + WhiteKernel(noise_level=0.00517)


In [6]:
full_body_slip_gp.load_params('../data/ral2023_dataset/warthog_tracks/boreal_mud/trained_params/slip/gp/steady-state/')

In [ ]:
full_body_slip_gp.predict_slip_from_body_vels()